In [ ]:
!pip install -U qdrant-client
!pip install llama-index
import os
from pathlib import Path
import pandas as pd
from llama_index import GPTVectorStoreIndex
from llama_index import ServiceContext
from llama_index import SimpleDirectoryReader
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

In [2]:
output_directory = "outputdirectory"

In [3]:
# I have sent the api keys which were used by me in the mail these can also be created by visiting the websites of openai and qdrant
os.environ['OPENAI_API_KEY'] = "YOUR_OPENAI_API"
os.environ['QDRANT_API_KEY'] = "YOUR_QDRANT_API"
client = QdrantClient(
    url="QDRANT_URL",
    api_key="QDRANT_API",
)
service_context = ServiceContext.from_defaults()
vector_store = QdrantVectorStore(client=client, collection_name="BigBasket")

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
# PLEASE UPDATE THIS WITH YOUR LOCATION OF DATASET
df=pd.read_csv("/content/bigBasketProducts.csv")

In [5]:
def list_files(directory):
    return list(os.scandir(directory))
Path.ls = list_files
files_dictionary = {row['product']: {column: row[column] for column in df.columns if column != 'product'} for _, row in df.iterrows()}
if not os.path.exists(output_directory):
    os.mkdir(output_directory)
for key in files_dictionary.keys():
    items = files_dictionary[key]
    key_str = str(key)
    file_path = f"{output_directory}/" + f"{key_str.replace('/','*')}.txt"
    with open(file_path, "w") as myfile:
        for item1, item2 in items.items():
            myfile.write(f"{item1}:{item2}\n")
output_directory = Path(output_directory).resolve()

In [6]:
def generate_metadata(file_name: str):
    date_str = Path(file_name).stem.split("*")[1:4]
    return {"product": ":".join(date_str)}
products = SimpleDirectoryReader(file_metadata=generate_metadata, input_files=output_directory.ls()).load_data()

In [7]:
#THIS WOULD TAKE A CONSIDERABLE TIME TO RUN PLEASE USE AN VALID OPEN AI API WHICH CAN PROCESS THIS MUCH DATA
#IF IT TAKES TO LONG to see if it works you can use products = products[:50] to limit the number of entries from table that goes to model
#it will always answer from that context only, ideally if you have the limit on your account we can remove products=products[:50] and the
#answers will come from entire dataset
products = products[:50]
ranks = GPTVectorStoreIndex.from_documents(products, vector_store=vector_store, service_context=service_context)

In [8]:
model = ranks.as_query_engine()

In [9]:
#PLEASE ENTER YOUR QUERY HERE :
response = model.query("what is the highest cost in snacks")
print(response)

The highest cost in snacks is 70.0.
